In [1]:
from PIL import Image
import numpy as np
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset
from torchvision import models

device = torch.device("cuda:0")

In [2]:
list_img = []
labels = []

directory = '../../new_data_norm2/'
for name in os.listdir(directory + 'yes'):
    img = Image.open(directory + 'yes/'+ name)
    img = img.resize((240,240))
    list_img.append(np.asarray(img))
    labels.append(1)
    
for name in os.listdir(directory + '/no'):
    img = Image.open(directory + 'no/'+ name)
    img = img.resize((240,240))
    list_img.append(np.asarray(img))
    labels.append(0)


In [3]:
class brain_Dataset(Dataset):
    def __init__ (self, data, transform=None):
        self.data = data
        self.transform = transform
        
    def __len__(self):
        return len(self.data[1])
        
    def __getitem__(self, index):
        img = self.data[0][index]
        label = self.data[1][index]
        
        if self.transform:
            img = self.transform(img)
        
        return img, label

In [4]:
transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])                           
                       ])

In [5]:
dataset = brain_Dataset([list_img, labels], transform)

In [6]:
batch_size = 15 ## A diminuer si cuda out of memory

data_size = len(list_img)
validation_split = .2
split = int(np.floor(validation_split * data_size))
indices = list(range(data_size))
np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

trainloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
valloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         sampler=val_sampler)

In [7]:
def train_model(model, train_loader, val_loader, loss, optimizer, num_epochs):    
    loss_history = []
    train_history = []
    val_history = []
    
    for epoch in range(num_epochs):
        model.train() # Enter train mode
        
        loss_accum = 0
        correct_samples = 0
        total_samples = 0
        for i_step, (x, y) in enumerate(train_loader):
            x_gpu = x.to(device, dtype=torch.float)
            y_gpu = y.to(device, dtype=torch.long)
            prediction = model(x_gpu)   
            loss_value = loss(prediction, y_gpu)
            optimizer.zero_grad()
            loss_value.backward()
            optimizer.step()
            
            _, indices = torch.max(prediction, 1)

            correct_samples += torch.sum(indices == y_gpu)
            total_samples += y.shape[0]
            
            loss_accum += loss_value

        ave_loss = loss_accum / i_step
        train_accuracy = float(correct_samples) / total_samples
        val_accuracy = compute_accuracy(model, val_loader)
        
        loss_history.append(float(ave_loss))
        train_history.append(train_accuracy)
        val_history.append(val_accuracy)
        
        print("Average loss: %f, Train accuracy: %f, Val accuracy: %f" % (ave_loss, train_accuracy, val_accuracy))
        
    return loss_history, train_history, val_history
        
def compute_accuracy(model, loader):
    model.eval() # Evaluation mode
    
    correct_samples = 0
    total_samples = 0
    
    for i_step, (x, y) in enumerate(loader):
      x_gpu = x.to(device, dtype=torch.float)
      y_gpu = y.to(device, dtype=torch.long)
      predictions = model(x_gpu)
      _, indices = torch.max(predictions, 1)
      correct_samples += torch.sum(indices == y_gpu)
      total_samples += y.shape[0]
      
    accuracy = float(correct_samples)/total_samples       
    
    return accuracy

### https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

### Resnet18

#### Fine-tuning

In [30]:
model = models.resnet18(pretrained=True)

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

model = model.to(device)

In [31]:
loss = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [32]:
train_model(model, trainloader, valloader, loss, optimizer, 5)

Average loss: 0.406963, Train accuracy: 0.842273, Val accuracy: 0.854220
Average loss: 0.204784, Train accuracy: 0.926564, Val accuracy: 0.961637
Average loss: 0.145642, Train accuracy: 0.945722, Val accuracy: 0.915601
Average loss: 0.114526, Train accuracy: 0.964240, Val accuracy: 0.951407
Average loss: 0.069807, Train accuracy: 0.977650, Val accuracy: 0.953964


([0.4069632291793823,
  0.204783633351326,
  0.14564168453216553,
  0.11452613025903702,
  0.06980697065591812],
 [0.8422733077905492,
  0.9265644955300127,
  0.9457215836526182,
  0.9642401021711366,
  0.9776500638569604],
 [0.8542199488491049,
  0.9616368286445013,
  0.9156010230179028,
  0.9514066496163683,
  0.9539641943734015])

#### Fixed feature extractor

In [24]:
res_mod = models.resnet18(pretrained=True)
for param in res_mod.parameters():
    param.requires_grad = False

num_ftrs = res_mod.fc.in_features
res_mod.fc = nn.Linear(num_ftrs, 2)

res_mod = res_mod.to(device)

In [25]:
loss = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor)
optimizer = optim.SGD(res_mod.parameters(), lr=1e-3)

In [26]:
train_model(res_mod, trainloader, valloader, loss, optimizer, 5)

Average loss: 0.682526, Train accuracy: 0.624521, Val accuracy: 0.659847
Average loss: 0.570795, Train accuracy: 0.722222, Val accuracy: 0.708440
Average loss: 0.526917, Train accuracy: 0.768199, Val accuracy: 0.744246
Average loss: 0.494851, Train accuracy: 0.782886, Val accuracy: 0.780051
Average loss: 0.470306, Train accuracy: 0.795658, Val accuracy: 0.782609


([0.6825255751609802,
  0.570794939994812,
  0.5269172787666321,
  0.4948510229587555,
  0.47030580043792725],
 [0.6245210727969349,
  0.7222222222222222,
  0.7681992337164751,
  0.7828863346104725,
  0.7956577266922095],
 [0.659846547314578,
  0.7084398976982097,
  0.7442455242966752,
  0.7800511508951407,
  0.782608695652174])

### Alexnet

In [20]:
model = models.alexnet(pretrained=True)

model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

model = model.to(device)

In [21]:
loss = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor)
optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [22]:
train_model(model, trainloader, valloader, loss, optimizer, 5)

Average loss: 0.446798, Train accuracy: 0.786718, Val accuracy: 0.851662
Average loss: 0.255177, Train accuracy: 0.894636, Val accuracy: 0.915601
Average loss: 0.159374, Train accuracy: 0.943167, Val accuracy: 0.928389
Average loss: 0.128514, Train accuracy: 0.954662, Val accuracy: 0.943734
Average loss: 0.103591, Train accuracy: 0.965517, Val accuracy: 0.969309


([0.4467983841896057,
  0.25517722964286804,
  0.15937399864196777,
  0.12851357460021973,
  0.10359135270118713],
 [0.7867177522349936,
  0.8946360153256705,
  0.9431673052362708,
  0.954661558109834,
  0.9655172413793104],
 [0.8516624040920716,
  0.9156010230179028,
  0.928388746803069,
  0.9437340153452686,
  0.969309462915601])